In [1]:
import pickle, gzip, numpy, urllib.request, json, boto3, sagemaker

In [2]:
urllib.request.urlretrieve("http://deeplearning.net/data/mnist/mnist.pkl.gz",'mnist.pkl.gz')
with gzip.open('mnist.pkl.gz','rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

In [3]:
from sagemaker import KMeans
from sagemaker import get_execution_role

In [4]:
prefix = 'sagemaker-kmeans-test'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}
my_region = boto3.session.Session().region_name
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")

TypeError: must be str, not NoneType

In [5]:
role = get_execution_role()

ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [ ]:
bucket_name = 'narayani1'
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)

In [ ]:
data_location = 's3://{}/sagemaker/kmeans_highlevel_example/data'.format(bucket_name)
output_location = 's3://{}/sagemaker/kmeans_example/output'.format(bucket_name)

In [ ]:
kmeans = KMeans(role=role, train_instance_count=2, train_instance_type='ml.c4.8xlarge',
               output_path=output_location, k=10, data_location=data_location)


In [ ]:
kmeans.fit(kmeans.record_set(train_set[0]))

In [ ]:
kmeans_predictor = kmeans.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [ ]:
result = kmeans_predictor.predict(train_set[0][30:31])
result

In [6]:
sagemaker.Session().delete_endpoint(kmeans_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

ValueError: Must setup local AWS configuration with a region supported by SageMaker.